## 앙상블 학습 : 보팅 방식 예제

### 보팅(Voting)
- 여러 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식  
- 일반적으로 서로 다른 알고리즘을 가진 분류기를 결합  

### 보팅 방식의 앙상블 예제 
- 로지스틱 회귀와 K-최근접 이웃 개별 알고리즘 사용  
- 유방암 데이터 세트 예측 분석 
- 보팅 방식으로 결합하고 성능 비교

### 유방암 데이터 세트 예측 모델
- 사용 데이터 세트 : 위스콘신 유방암 데이터 세트
- 데이터 세트 분리 : 테스트용(평가용) 데이터 세트 20%
- 사용하는 ML 알고리즘   
    - 로지스틱 회귀와 KNN 알고리즘을 보팅 방식으로 결합   
- 모델 평가 : 예측 성능 평가 - 정확도  
    - 개별 알고리즘 정확도  
        - LogisticRegression 정확도   
        - KNeighborsClassifier 정확도  
    - Voting 분류기 정확도 

### 보팅 분류기 생성 
- 보팅 방식의 앙상블을 구현한 사이킷런의 VotingClassifier 클래스 이용
- from sklearn.ensemble import VotingClassifier  
- VotingClassifier(estimators, voting)    
    - estimators
        - 리스트 값
        - 보팅에 사용될 여러 개의 Classifier 객체들을 튜플 형식으로 입력 받음    
        - [('LR', lr_clf),('KNN', knn_clf)]    
    - voting : 보팅 방식 (hard/soft)(디폴트:hard) 
        - hard : 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정  
        - soft : 분류기들이 레이블 값 결정 확률을 평균내서 확률이 가장 높은 레이블 값 선정  

### 유방암 데이터 세트 예측 프로세스
- (1) 데이터 세트 준비 : 위스콘신 유방암 데이터 세트
    - load_breast_cancer()
- (2) 데이터 세트 분리 : 학습 데이터 / 테스트 데이터 세트
- (3) 보팅 분류기 생성 
    - 개별 알고리즘 
    - 보팅 분류기
- (4) 모델 학습 / 예측 / 평가
    - 개별 알고리즘
    - 보팅 분류기

위스콘신 유방암 데이터 세트  
- 유방암의 악성종양, 양성종양 여부를 결정하는 이진 분류 데이터 세트  
- 종양의 크기, 모양 등의 형태와 관련한 많은 피처 포함  
- load_breast_cancer() 함수를 통해 위스콘신 유방암 데이터 세트 생성  
- label이 
    - 0 : malignant(악성 종양)   
    - 1 : benign(양성:정상)  

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [1]:
# threadpoolctl 버전 확인
import threadpoolctl
threadpoolctl.__version__

'3.1.0'

In [18]:
# 버전 변경 
# !pip install threadpoolctl==3.1.0

  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0


In [1]:
import threadpoolctl
threadpoolctl.__version__
# 주피터 노트북 재시작

'3.1.0'

### (1) 데이터 세트 준비 : 위스콘신 유방암 데이터 세트

In [3]:
import pandas as pd


from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.tail(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
568,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


### (2) 데이터 세트 분리 : 학습 데이터 / 테스트 데이터 세트

In [4]:
from sklearn.model_selection import train_test_split

# 데이터 세트 분리
X_train, X_test, y_train, y_test = train_test_split(cancer.data, 
                                                    cancer.target, 
                                                    test_size=0.2, 
                                                    random_state=156)

### (3) 보팅 분류기 생성 
- 개별 알고리즘 
- 보팅 분류기

In [6]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [7]:
# (1) 개별 알고리즘 분류기 생성
# 로지스틱 회귀와 KNN 사용
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8) # 이웃 수를 8로

In [10]:
# (2) 보팅 분류기 생성
# 소프트 보팅
# estimators 2개 : 리스트 형태 각 분류기에 이름을 붙임
# lr_clf : LR
# knn_clf : KNN
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)],
                         voting='soft')

In [ ]:
# 모델 = 분류기 = estimator

### (4) 모델 학습 / 예측 / 평가 수행  
- Voting 분류기 학습 / 예측 / 평가
- 개별 알고리즘 학습 / 예측 / 평가
- 결과 비교

In [12]:
# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
y_pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

VotingClassifier(estimators=[('LR', LogisticRegression(solver='liblinear')),
                             ('KNN', KNeighborsClassifier(n_neighbors=8))],
                 voting='soft')

Voting 분류기 정확도 : 0.9561


In [13]:
# 로지스틱 회귀와 KNN 각 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    clf_name = clf.__class__.__name__ # 내장 속성 (클래스 이름)
    print('{0} 정확도 : {1:.4f}'.format(clf_name, accuracy_score(y_test, y_pred)))

LogisticRegression(solver='liblinear')

LogisticRegression 정확도 : 0.9474


KNeighborsClassifier(n_neighbors=8)

KNeighborsClassifier 정확도 : 0.9386


In [14]:
# 결과
# 보팅 분류기의 정확도가 약간 높게 나타났고
# 로지스틱 회귀, KNN순으로 나왔음

# 보팅으로 여러개의 분류기를 결합한다고 해서 
# 무조건 예측 성능이 많이 향상되지는 않음

# 그래도
# 보팅, 배깅, 부스팅 등의 앙상블 방법은
# 전반적으로 다른 단일 ML 알고리즘 사용하는 경우보다 
# 뛰어난 예측 성능 가지는 경우가 많음


# 고정된 데이터 세트에서
# 단일 ML 알고리즘이 뛰어난 성능을 발휘하더라도
# 현실 세계는 다양한 변수, 예측 어려운 규칙으로 구성되어있기 때문에
# 다양한 관점을 가지는 알고리즘이 서로 결합해서
# 더 나은 성능을 실제 환경에서 이끌어 낼 수 있다.

# 단일 알고리즘 뿐만 아니라
# 여러 알고리즘을 결합하는 방식을 같이 사용해서
# 최적의 알고리즘을 찾는 방식으로 진행

### 참고
- ML 모델의 성능은 다양한 테스트에 의해 검증되므로
- 어떻게 높은 유연성을 가지고 현실에 대처할 수 있는 가가 
- 중요한 ML 모델의 평가 요소가 됨

---
- 보팅과 스태킹은 서로 다른 알고리즘을 기반으로 하고 있지만
- 배깅과 부스팅은 대부분 결정 트리 알고리즘을 기반으로 하고 있음 
- (다른 알고리즘을 사용해서는 안 된다는 의미는 아님)

- 결정트리 알고리즘은 쉽고 직관적인 기준을 가지고 있지만
- 정확한 예측을 위해 학습 데이터를 과하게 학습해서
- 과적합이 발생해서
- 실제 테스트 데이터에서 예측 성능이 떨어지는 현상이 발생하기 쉬움

---
- 그러나 앙상블 학습에서는 수십~수천 개의 많은 분류기를 결합해서 
- 다양한 상황을 학습하게함으로써 
- 결정 트리 알고리즘의 장점은 그대로 취하고
- 단점은 보완하면서 편향-분산 트레이드 오프의 효과를 극대화할 수 있음 

---
편향-분산 트레이드 오프
- 편향 : 한쪽으로 치우치는 결과  
- 분산 : 데이터의 퍼짐 정도  
- 모형이 복잡하면 분산은 증가할 것이고  
- 모형이 너무 간단하면 편향이 발생하게 됨  
- 반비례 특성이기 때문에  
- 이 둘 간에 적절한 선택을 해야 함   
- 일반적으로 편향은 적고 분산은 약간 높은 이런 모형이 선호된다고 할 수 있음    


